In [24]:
from Bio import SeqIO
from os import listdir

##### 1. Get reads tagged as prophages

In [65]:
fw = open("extractFastq.prophages.list.txt",'w')
folder = "/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp/2_readsCleaning/9.readsForDepletion.bacteria"
for f in listdir(folder): #f:=file
    if "prophages" in f:
        sample = f.split('.')[0]
        fordward = "8L_PF_scythed_tagcleaned_sickled_"+sample+"_paired_R1.fastq"
        reverse = "8L_PF_scythed_tagcleaned_sickled_"+sample+"_paired_R2.fastq"
        folderFastq = "/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp/0_readsVirome/"
        outfileR1 = sample+".prophages.R1.fastq"
        outfileR2 = sample+".prophages.R2.fastq"
        #extractFromFastQ(folderFastq+fordward,folder+'/'+f,outfile)
        print >> fw, "python /ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/code/extractFromFastQ.py " + \
                 folderFastq+fordward + " " + \
                 folder+'/'+f + " " + \
                 outfileR1
        #extractFromFastQ(folderFastq+reverse,folder+'/'+f,outfile)
        print >> fw, "python /ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/code/extractFromFastQ.py " + \
                 folderFastq+ reverse + " " + \
                 folder+'/'+f + " " + \
                 outfileR2
fw.close()

In [66]:
%%writefile callRunExtractFastq.sh
cat extractFastq.prophages.list.txt | while read -r line
do
    qsub -v FOO="$line" runExtractFastq.sh
done

Overwriting callRunExtractFastq.sh


In [77]:
%%writefile runExtractFastq.sh
#!/bin/bash
#$ -N eFQ
#$ -l h_vmem=8G
#$ -l h_rt=00:10:00
#$ -cwd
#$ -m ea
#$ -M jmoreno@tuebingen.mpg.de

export PATH=/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/anaconda3/bin:$PATH
source activate py2_leo

line=$FOO
eval $line

Overwriting runExtractFastq.sh


In [70]:
%%bash
chmod +x callRunExtractFastq.sh
./callRunExtractFastq.sh

##### 2. Get  reads tagged as contaminants

In [75]:
fw = open("extractFastq.contaminants.list.txt",'w')
folder = "/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp/2_readsCleaning/9.readsForDepletion.bacteria"
for f in listdir(folder): #f:=file
    if "contaminants" in f:
        sample = f.split('.')[0]
        fordward = "8L_PF_scythed_tagcleaned_sickled_"+sample+"_paired_R1.fastq"
        reverse = "8L_PF_scythed_tagcleaned_sickled_"+sample+"_paired_R2.fastq"
        folderFastq = "/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp/0_readsVirome/"
        outfileR1 = sample+".contaminants.R1.fastq"
        outfileR2 = sample+".contaminants.R2.fastq"
        #extractFromFastQ(folderFastq+fordward,folder+'/'+f,outfile)
        print >> fw, "python /ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/code/extractFromFastQ.py " + \
                 folderFastq+fordward + " " + \
                 folder+'/'+f + " " + \
                 outfileR1
        #extractFromFastQ(folderFastq+reverse,folder+'/'+f,outfile)
        print >> fw, "python /ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/code/extractFromFastQ.py " + \
                 folderFastq+ reverse + " " + \
                 folder+'/'+f + " " + \
                 outfileR2
fw.close()

In [76]:
%%writefile callRunExtractFastq.sh
cat extractFastq.contaminants.list.txt | while read -r line
do
    qsub -v FOO="$line" runExtractFastq.sh
done

Overwriting callRunExtractFastq.sh


In [79]:
%%bash
chmod +x callRunExtractFastq.sh
./callRunExtractFastq.sh

In [80]:
#Move files to tmp2 folder


In [81]:
%%bash
mkdir ../../tmp2/10_contaminants
mkdir ../../tmp2/10_contaminants/readsTaggedAsProphagesOrContaminants
mv *.fastq ../../tmp2/10_contaminants/readsTaggedAsProphagesOrContaminants

##### 3. Map reads against NT viral

In [102]:
fw1 = open("phrophagesVsNTviral.list.txt",'w')
fw2 = open("contaminantsVsNTviral.list.txt",'w')
folder = "/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/10_contaminants/readsTaggedAsProphagesOrContaminants/"
for f in listdir(folder):
    if "R1" in f:
        sample = f.split('.')[0]
        if "prophages" in f:
            print >> fw1, "/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/Programs/centrifuge/centrifuge -p 15 -q "+\
            "-x /ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/0X_contigAnnotation/centrifugeDatabase/viral " +\
            "--report-file "+sample+".prophages.centrifuge.report.tsv "+\
            "-1 "+folder+f + " " +\
            "-2 "+folder+f.replace("R1","R2")+" > "+sample+".prophages.centrifuge.viral.tsv" 
        if "contaminants" in f:
            print >> fw2, "/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/Programs/centrifuge/centrifuge -p 15 -q "+\
            "-x /ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/0X_contigAnnotation/centrifugeDatabase/viral " +\
            "--report-file "+sample+".contaminants.centrifuge.report.tsv "+\
            "-1 "+folder+f + " " +\
            "-2 "+folder+f.replace("R1","R2")+" > "+sample+".contaminants.centrifuge.viral.tsv"
fw1.close()
fw2.close()

In [94]:
#As centrifuge is very fast, I decided to sumbmit all samples in serial not in parallel. 
#I Add the header to each file (phrophagesVsNTviral.list.txt and contaminantsVsNTviral.list.txt) 

###HEADER###
#!/bin/bash
#$ -N centrifugePhages
#$ -pe parallel 50
#$ -l h_vmem=8G
#$ -l h_rt=05:00:00
#$ -cwd
#$ -m ea
#$ -M jmoreno@tuebingen.mpg.de

In [103]:
%%bash 
qsub contaminantsVsNTviral.list.txt
qsub phrophagesVsNTviral.list.txt

Your job 3165534 ("centrifugePhages") has been submitted
Your job 3165535 ("centrifugePhages") has been submitted


In [107]:
%%bash
mkdir ../../tmp2/10_contaminants/centrifugeViralOutput
mv *.tsv ../../tmp2/10_contaminants/centrifugeViralOutput

#### 4. Map reads against NT

In [115]:
#Download all NT centrifuges database, that includes all bacteria,archaea,and viral seqs in the NCBI

In [118]:
#I run this command directly in the terminal using screen as it will take a while
# being in this forlder: /ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/0X_contigAnnotation
#nohup /ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/Programs/centrifuge/centrifuge-download -o library2 -m -d "archaea,bacteria,viral" refseq > seqid2taxid.map &
# Move the files to the database folder
# cd centrifugeDatabase 
# mv ../library2/
# mv ../seqid2taxid.map seqid2taxid.library2.map


# cat archaea.fna viral.fna bacteria.fna > input-sequences.abv.fna
#/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/Programs/centrifuge/centrifuge-build -p 10 --conversion-table seqid2taxid.library2.map --taxonomy-tree taxonomy/nodes.dmp --name-table taxonomy/names.dmp input-sequences.abv.fna abv


In [119]:
fw1 = open("phrophagesVsNT.list.txt",'w')
fw2 = open("contaminantsVsNT.list.txt",'w')
folder = "/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/10_contaminants/readsTaggedAsProphagesOrContaminants/"
for f in listdir(folder):
    if "R1" in f:
        sample = f.split('.')[0]
        if "prophages" in f:
            print >> fw1, "/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/Programs/centrifuge/centrifuge -p 15 -q "+\
            "-x /ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/0X_contigAnnotation/centrifugeDatabase/abv " +\
            "--report-file "+sample+".prophages.centrifuge.report.tsv "+\
            "-1 "+folder+f + " " +\
            "-2 "+folder+f.replace("R1","R2")+" > "+sample+".prophages.centrifuge.abv.tsv" 
        if "contaminants" in f:
            print >> fw2, "/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/Programs/centrifuge/centrifuge -p 15 -q "+\
            "-x /ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/0X_contigAnnotation/centrifugeDatabase/abv " +\
            "--report-file "+sample+".contaminants.centrifuge.report.tsv "+\
            "-1 "+folder+f + " " +\
            "-2 "+folder+f.replace("R1","R2")+" > "+sample+".contaminants.centrifuge.abv.tsv"
fw1.close()
fw2.close()

In [ ]:
#As centrifuge is very fast, I decided to sumbmit all samples in serial not in parallel. 
#I Add the header to each file (phrophagesVsNTviral.list.txt and contaminantsVsNTviral.list.txt) 

###HEADER###
#!/bin/bash
#$ -N centrifugePhages
#$ -pe parallel 50
#$ -l h_vmem=8G
#$ -l h_rt=05:00:00
#$ -cwd
#$ -m ea
#$ -M jmoreno@tuebingen.mpg.de

In [120]:
%%bash 
qsub contaminantsVsNTviral.list.txt
qsub phrophagesVsNTviral.list.txt

In [121]:
%%bash
mkdir ../../tmp2/10_contaminants/centrifugeABVOutput
mv *.tsv ../../tmp2/10_contaminants/centrifugeABVOutput

#### 5. Map reads against NR

In [108]:
fw1 = open("phrophagesVsNR.list.txt",'w')
fw2 = open("contaminantsVsNR.list.txt",'w')
folder = "/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/10_contaminants/readsTaggedAsProphagesOrContaminants/"
for f in listdir(folder):
    if "prophages" in f:
        print >> fw1, "/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/Programs/diamond/diamond blastx "+\
        "--threads 10 " +\
        "-d /ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/0X_contigAnnotation/ncbiNRdatabase/nrDIAMOND " +\
        "-q "+folder+f + " " +\
        "--daa "+f.replace("fastq","daa")+ " " +\
        "--max-target-seqs 25 --evalue 1e-5"
    if "contaminants" in f:
        print >> fw2, "/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/Programs/diamond/diamond blastx "+\
        "--threads 10 " +\
        "-d /ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/0X_contigAnnotation/ncbiNRdatabase/nrDIAMOND " +\
        "-q "+folder+f + " " +\
        "--daa "+f.replace("fastq","daa")+ " " +\
        "--max-target-seqs 25 --evalue 1e-5"
fw1.close()
fw2.close()

In [ ]:
%%writefile callRunDiamond.sh
cat phrophagesVsNR.list.txt | while read -r line
do
    qsub -v FOO="$line" runDiamond.sh
done

In [112]:
%%writefile callRunDiamond.sh
cat contaminantsVsNR.list.txt | while read -r line
do
    qsub -v FOO="$line" runDiamond.sh
done

Overwriting callRunDiamond.sh


In [110]:
%%writefile runDiamond.sh
#!/bin/bash
#$ -N diamondP
#$ -l h_vmem=8G
#$ -pe parallel 10
#$ -l h_rt=02:00:00
#$ -cwd
#$ -m ea
#$ -M jmoreno@tuebingen.mpg.de

line=$FOO
eval $line

Writing runDiamond.sh


In [114]:
%%bash
chmod +x callRunDiamond.sh
./callRunDiamond.sh

In [122]:
%%bash
mkdir ../../tmp2/10_contaminants/diamondOutput
mv *.daa ../../tmp2/10_contaminants/diamondOutput